## About WindTrace
WindTrace is a python open-source parametric life-cycle inventory model to create tailor-made inventories of onshore wind turbines and park fleets and assess their environmental impacts. \
More information on GitHub: https://github.com/LIVENlab/WindTrace_public

Copyright (c) 2025 LIVEN lab \
Licensed under the MIT License. See LICENSE for details.

## WindTrace step-by-step

### Outlook
#### 1. Create environment and install the requirements
#### 2. Set consts.py variables
#### 3. Create/open a brightway project
#### 4. Creating life-cycle inventories
##### 4.1 Example of application
##### 4.2 Exploring the inventories
##### 4.3 Calculating LCIA scores
##### 4.4 Another example with land use focus
##### 4.5 Creating a future wind park
#### 5. Importing example turbine datasets

### 1. Create environment and install the requirements

Open the conda console and run the following line. Be sure you are in the folder where the environment.yml file is saved.
```ruby
conda env create -f environment.yml 
```
This will create a new environment called 'windtrace_env' with all the packages you need to run this Jupyter Notebook already installed. \
Make sure to activate this environment before running this Notebook.

### 2. Set consts.py variables

**IMPORTANT**: Before running anything, in the file consts.py set the following variables: PROJECT_NAME, SPOLD_FILES, and NEW_DB_NAME.
- PROJECT_NAME: name of the brightway project to be used.
- SPOLD_FILES: folder path where you have the Spoldfiles from Ecoinvent in your local disk.
- NEW_DB_NAME: name of the databes where all the new wind turbine inventories you creeated will be stored.

### 3. Create/open a brightway project

In [1]:
import bw2data as bd
import bw2io as bi
import consts
from WindTrace_onshore import *

In [2]:
bd.projects.set_current(consts.PROJECT_NAME)
bi.bw2setup()

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


100%|███████████████████████████████████████████████████████████████████████████| 4709/4709 [00:00<00:00, 25047.62it/s]


Vacuuming database 
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations



In [4]:
spold_files = consts.SPOLD_FILES
if "cutoff391" not in bd.databases:
    ei = bi.SingleOutputEcospold2Importer(spold_files, "cutoff391", use_mp=False)
    ei.apply_strategies()
    ei.write_database()
cutoff391 = bd.Database("cutoff391")
if consts.NEW_DB_NAME not in bd.databases:
    new_db = bd.Database(consts.NEW_DB_NAME)
    new_db.register()
new_db = bd.Database(consts.NEW_DB_NAME)
biosphere3 = bd.Database('biosphere3')

100%|████████████████████████████████████████████████████████████████████████████| 21238/21238 [06:18<00:00, 56.13it/s]


Extracted 21238 datasets in 381.08 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: convert_activity_parameters_to_list
App

100%|███████████████████████████████████████████████████████████████████████████| 21238/21238 [00:50<00:00, 421.06it/s]


Vacuuming database 
Created database: cutoff391


### 4. Creating life-cycle inventories

The function allowing to create customized inventories is **lci_wind_turbine()**, which admits the following parameters:
| Parameter                   | Description                                                  | Default value                                   | Type                                                           |
|-----------------------------|--------------------------------------------------------------|-------------------------------------------------|----------------------------------------------------------------|
| Park name                   | Name of the wind park                                        | No                                              | str                                                            |
| Park power                  | Total power of the wind park [MW]                            | No                                              | float                                                          |
| Number of turbines          | Number of turbines in the wind park                          | No                                              | int                                                            |
| Park location               | Abbreviation of the country (ISO 3166-1 alpha-2 codes)       | No                                              | str                                                            |
| Park coordinates            | WGS84 coordinates of the wind park (latitude, longitude)     | No                                              | tuple                                                          |
| Manufacturer                | Name of the turbine manufacturer     | LM Wind                                         | str [Literal['LM Wind','Vestas','Siemens Gamesa','Enercon','Nordex']                   |
| Rotor diameter              | Diameter of the rotor (in meters)                            | No                                              | int                                                            |
| Turbine rated power         | Nominal power of the individual turbines [MW]                | No                                              | float                                                          |
| Hub height                  | Height of the turbine (in meters)                           | No                                              | float                                                          |
| Regression adjustment       | Calculate mass of steel as a function of 'D2h' or 'hub height'| D2h                                            | Literal['D2h', 'Hub height']
 |
| Commissioning year          | Year when the turbine started operation                      | No                                              | int                                                            |
| Recycled steel share        | Recycled content share of steel                       | Data from Eurofer (2012-2021)                   | float                                                          |
| Land use permanent intensity | Permanent direct land transformation per MW                 | 3000 m²/MW                                      | float                                                          |
| Electricity mix steel       | Country of the electricity mix for steel production                         | Mix per Eurofer data by country                 | Optional[Literal['Norway', 'Europe', 'Poland']]                |
| Generator type              | Gearbox and type of generator                                | Doubly fed induction generator (gearbox)        | Optional[Literal['dd_eesg', 'dd_pmsg', 'gb_pmsg', 'gb_dfig']]  |                                                           |
| Land cover type             | Land cover type prior to turbine installation                | No                                              | str[Literal['industrial, from','crops, non-irrigated','unspecified', 'row crops', 'shrubland', 'grassland', 'pasture', 'forest', 'industrial','road']                                              |
| End-of-life scenario        | Scenario based on material recycling rates (1: baseline, 2: optimistic, 3: pessimistic)                   | Baseline                                        | str                                                            |
| Lifetime                    | Expected lifetime of the turbine (years)                            | 20 years                                        | int                                                            |
| Capacity factor             | Ratio of average delivered electricity to maximum theoretical production | 0.24                                            | float                                                          |
| Attrition rate              | Annual efficiency reduction due to wear                      | 0.009                                           | float                                                          |

Parameters without a default value must be filled. Parameters with a default value are not compulsory but can be adapted to the user needs.

### 4.1. Example of application

Let's build an imaginary 100 MW wind park ('my_wind_park') of 20 turbines, 5 MW each, in a random location in Spain. \
The manufacturer of the turbine is Siemens Gamesa, with turbines of 130 m rotor diameter, 140 m hub height and a gear-boxed double-fed induction generation drivetrain. \
The park is commissioned in 2020 and is expected to last for 25 years. \
The expected initial capacity factor is 24%. \
**IMPORTANT**: make sure the name you give to the park (park_name variable) has not been used previously in your database! This means, you cannot create two wind parks with the exact same name in the same database.

In [7]:
lci_wind_turbine(park_name='my_wind_park', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',
                 lifetime=25,  
                 cf=0.24,
                 new_db=new_db,
                 biosphere3=biosphere3,
                 cutoff391=cutoff391
                )

WARNING. You did not select any electricity_mix. The mean shares by country applied in the steel industry between 2017 and 2021 will be used
Electricity mix: European mix provided by Eurofer


ValidityError: This activity can't be saved for the following reasons
	* ``database`` refers to unknown database

### 4.2 Exploring the inventories

The inventories are given by unit of turbine, unit of park, and kWh (for both turbine and park). In this example, the inventoris would have the following names (and codes):

- Turbine (FU=unit): my_wind_park_single_turbine
- Turbine (FU=kWh): my_wind_park_turbine_kwh
- Park (FU=unit): my_wind_park_10.0
- Park (FU=kWh): my_wind_park_kwh

You can access them as follows:

In [12]:
# park (FU=unit)
park_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_100.0')
park_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_park_kwh')
turbine_unit_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_single_turbine')
turbine_kwh_act = bd.Database(consts.NEW_DB_NAME).get('my_wind_park_turbine_kwh')

### 4.3 Calculating LCIA scores

LCA impacts of a park or an individual turbine of the park can be calculated with lca_wind_turbine(). It returns two dictionaries: the first one being results per unit and the second one being results per kWh. Default results are given per turbine. For the whole park, you must set the parameter 'turbine' to False. ReCiPe 2016 v1.03, midpoint (H) is the default LCIA method. It can be manually changed by giving the parameter 'method' another name of a method in Brightway.

Here an example of application for the whole park. The first dictionary saves scores per FU=unit, and the second per FU=1kWh

In [13]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', turbine=False, new_db=new_db)

Functional unit: 1 wind park
LCIA methods: EF v3.1
CF: 0.24. Attrition rate: 0.009


Alternatively, you can give it a certain set of indicators of your choice as a list with the variable 'indicators'. Example:

In [14]:
lcia_scores_unit, lcia_scores_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT'), 
                                           ('Cumulative Energy Demand (CED)', 'energy resources: renewable', 'energy content (HHV)'),
                                           ('Ecological Scarcity 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
                                          ], 
                               turbine=False, new_db=new_db)

Functional unit: 1 wind park
LCIA methods: ('CML v4.8 2016 no LT', 'acidification no LT', 'acidification (incl. fate, average Europe total, A&B) no LT'), ('Cumulative Energy Demand (CED)', 'energy resources: renewable', 'energy content (HHV)'), ('Ecological Scarcity 2021 no LT', 'climate change no LT', 'global warming potential (GWP100) no LT')
CF: 0.24. Attrition rate: 0.009


### 4.4 Another example with land use focus

Now let's build another wind park changing a bit the land use related parameters. Let's say we know the original land where the turbine will be built is a pasture. Let's say we also know that the permanent land use intensity is smaller than the default number we use. This parameter should include the squared meters per MW of land that will be occupied by permanent access roads, transformer, substation and the turbines themselves. We will use an intensity of 1000 m2/MW this time.

In [15]:
lci_wind_turbine(park_name='my_wind_park_LandUse', 
                 park_power=100.0, 
                 number_of_turbines=20, 
                 park_location='ES', 
                 park_coordinates=(41.502, -1.126), 
                 manufacturer='Siemens Gamesa', 
                 rotor_diameter=130, 
                 turbine_power=5.0, 
                 hub_height=140, 
                 commissioning_year=2020, 
                 generator_type='gb_dfig',  
                 lifetime=25,  
                 cf=0.24,
                 land_use_permanent_intensity=1000,
                 land_cover_type='pasture',
                 new_db=new_db,
                 biosphere3=biosphere3,
                 cutoff391=cutoff391
                )

Time-adjusted cf applied with an attrition coefficient of 0.009


({'Low alloy steel': 7455354.805104657,
  'Chromium steel': 1043984.2519468605,
  'Cast iron': 1681063.4157545082,
  'Aluminium': 161845.10502487788,
  'Copper': 91393.59437802702,
  'Epoxy resin': 311104.78386133275,
  'Rubber': 27173.579357149145,
  'PUR': 50570.48514631777,
  'PVC': 135072.30653721892,
  'Fiberglass': 712338.5970491616,
  'electronics': 24041.93530003225,
  'Electrics': 76255.71393607548,
  'Lubricating oil': 31331.69045155677,
  'Low alloy steel_foundations': 2052452.3349240404,
  'Chromium steel_foundations': 171367.63623532938,
  'Concrete_foundations': 15677.101183848605,
  'Praseodymium': 0.0,
  'Neodymium': 1200.0,
  'Dysprosium': 200.0,
  'Terbium': 0.0,
  'Boron': 0.0,
  'PE': 1129.9114902604465},
 40000.0,
 212500.0)

And compare the land-use impacts with the previous wind park we created:

In [18]:
previous_park_score_unit, previous_park_score_kwh = lca_wind_turbine(park_name='my_wind_park', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
new_park_score_unit, new_park_score_kwh  = lca_wind_turbine(park_name='my_wind_park_LandUse', park_power=100.0, method='EF v3.1', 
                               indicators=[('EF v3.1', 'land use', 'soil quality index')
                                          ], 
                               turbine=False, new_db=new_db)
print(f'Land Use score previous park (FU=one park): {previous_park_score_unit}; (FU=1kWh): {previous_park_score_kwh}')
print(f'Land Use score new park(FU=one park): {new_park_score_unit}; (FU=1kWh): {new_park_score_kwh}')

Functional unit: 1 wind park
LCIA methods: ('EF v3.1', 'land use', 'soil quality index')
CF: 0.24. Attrition rate: 0.009
Functional unit: 1 wind park
LCIA methods: ('EF v3.1', 'land use', 'soil quality index')
CF: 0.24. Attrition rate: 0.009
Land Use score previous park (FU=one park): {'land use': 8093328991.7375345}; (FU=1kWh): {'land use': 1.7274932806129133}
Land Use score new park(FU=one park): {'land use': 5751582028.022938}; (FU=1kWh): {'land use': 1.2276554328196807}


### 4.5 Creating a future wind park

EXPLANATION

### 5. Importing example turbine datasets